# **INTRODUCTIONS**

---

Used for exceptional processes.

The code is separated into multiple sections:

1. **[Views](#views)**
2. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/dataflows/rivm_booster_shots_administered.ipynb",        
        "src/dataflows/rivm_booster_shots_coverage.ipynb",
        "src/dataflows/rivm_third_shots_administered.ipynb",
        "src/common/schemas.ipynb"
    ]
}
```

# **TABLES**

---


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
DROP TABLE IF EXISTS [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]

CREATE TABLE [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS](
	[ID] [INT] IDENTITY (1,1) NOT NULL,
	[CATEGORY] [NVARCHAR](255) NULL,
	[KEY] [NVARCHAR](255) NULL,
	[VALUE] [NVARCHAR](255) NULL,
  	[DATE_LAST_INSERTED] [DATETIME] NOT NULL DEFAULT GETDATE()
);
GO;

-- 2) CREATE STAGE INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_STAGE_MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS')
CREATE NONCLUSTERED INDEX [IX_STAGE_MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS] ON [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
(
    [DATE_LAST_INSERTED] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

# **VIEWS**
---

### BOOSTER SHOT ADMINISTERED

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  VIEW [VWSDEST].[V_MANUAL_BOOSTER_SHOT_ADMINISTERED]
AS
	WITH CTE AS (
		SELECT *
		FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
		AND CATEGORY = 'BOOSTER_SHOT_ADMINISTERED' 
	)
    SELECT
        -- Select columns,
        Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'GGD_ADMINISTERED_LAST_7_DAYS') AS [GGD_ADMINISTERED_LAST_7_DAYS] ,
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'GGD_ADMINISTERED_TOTAL') AS [GGD_ADMINISTERED_TOTAL],
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'OTHERS_ADMINISTERED_TOTAL') AS [OTHERS_ADMINISTERED_TOTAL],
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_START_UNIX'), 105)) AS [DATE_START_UNIX],
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_END_UNIX'), 105)) AS [DATE_END_UNIX],
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO;



### THIRD SHOTS ADMINISTERED

In [ ]:

-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  VIEW [VWSDEST].[V_MANUAL_THIRD_SHOT_ADMINISTERED]
AS
	WITH CTE AS (
		SELECT *
		FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
		AND CATEGORY = 'THIRD_SHOT_ADMINISTERED'
	)
    SELECT
        -- Select columns,
		Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'ADMINISTERED_TOTAL')  AS ADMINISTERED_TOTAL,
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS DATE_UNIX,
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO;



### BOOSTER AND THIRD SHOTS ADMINISTERED

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE  OR ALTER VIEW [VWSDEST].[V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED]
AS
	WITH CTE AS (
		SELECT 
			*
		FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
		AND CATEGORY = 'BOOSTER_AND_THIRD_SHOT_ADMINISTERED'
	)
    SELECT
        -- SELECT COLUMNS,
        Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'ADMINISTERED_TOTAL') AS ADMINISTERED_TOTAL,
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS DATE_UNIX,
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO;


### BOOSTER SHOTS PLANNED

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE  OR ALTER    VIEW [VWSDEST].[V_MANUAL_BOOSTER_SHOT_PLANNED]
AS
	WITH CTE AS (
		SELECT *
		FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
		AND CATEGORY = 'BOOSTER_SHOT_PLANNED'
	)
    SELECT
        -- SELECT COLUMNS,
        Top (1)
		(SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'PLANNED_7_DAYS') AS PLANNED_7_DAYS,
		dbo.[CONVERT_DATETIME_TO_UNIX] (CONVERT([DATETIME], (SELECT [VALUE] FROM CTE WHERE [KEY] = 'DATE_UNIX'), 105)) AS [DATE_UNIX],
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO;


### BOOSTER COVERAGE

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL]
AS
    WITH CTE AS (
        SELECT
            *
        FROM [VWSDEST].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS])
    )
    SELECT TOP(1)
        (SELECT CAST([VALUE] AS INT) FROM CTE WHERE [KEY] = 'BOOSTER_COVERAGE') AS [PERCENTAGE]
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_UNIX]
        dbo.[CONVERT_DATETIME_TO_UNIX] ((SELECT [DATE_LAST_INSERTED] FROM CTE WHERE [KEY] = 'DATE_UNIX')) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE
GO

# **DROP ARTIFACTS**

---


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) DROP VIEW(S).....
DROP VIEW IF EXISTS [VWSDEST].[V_Handmatig_BOOSTER_SHOT_PLANNED]
GO

DROP VIEW IF EXISTS [VWSDEST].[V_Handmatig_BOOSTER_AND_THIRD_SHOT_ADMINISTERED]
GO

DROP VIEW IF EXISTS [VWSDEST].[V_Handmatig_THIRD_SHOT_ADMINISTERED]
GO

DROP VIEW IF EXISTS [VWSDEST].[V_Handmatig_BOOSTER_SHOT_ADMINISTERED]
GO

-- 2) DROP TABLE(S).....
DROP TABLE IF EXISTS [VWSSTAGE].[Handmatige_invoer_booster_data]
GO


# **DATATINO CONFIGURATIONS**
---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT BOOSTER DATA'

-- 1) UPSERT WORKFLOW(S).....
EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = null, 
    @workflow_name = @workflow_name,
    @description = 'Manual insert of booster data', 
    @schedule = '0 13 * * TUE,THU', -- SET TO ACTIVE ONLY WHEN WORKFLOW NEEDS TO BE RUN
    @active = 1

-- 2) UPSERT SOURCE(S).....
EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE] 
    @id = null,
    @source_name = 'SOURCE_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    @description = 'Source manual insert of booster data',
    @source = 'datafiles/TEST/Handmatige_invoer_Booster_data.csv',
    @source_columns = 'CATEGORY|KEY|VALUE',
    @target_columns = 'CATEGORY|KEY|VALUE',
    @target_name = 'VWSSTAGE.MANUAL_INPUT_BOOSTER_AND_THIRD_SHOTS',
    @source_type = 'CsvFile',
    @location_type = 'AzureBlob',
    @delimiter_type = 'SemiColon',
    @security_profile= 'RIVM'
GO

### PROTOS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'MANUAL INSERT BOOSTER DATA'

-- 1) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_BOOSTER_SHOT_ADMINISTERED',
    @description = 'View of V_MANUAL_BOOSTER_SHOT_ADMINISTERED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null

-- 2) UPSERT PROTO CONFIGURATION(S).....

EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'View configuration for VWSDEST.V_MANUAL_BOOSTER_SHOT_ADMINISTERED',
    @view_name = 'VWSDEST.V_MANUAL_BOOSTER_SHOT_ADMINISTERED',
    @item_name = 'booster_shot_administered',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1'

--VIEW THIRD_SHOT_ADMINISTERED
-- 1) UPSERT PROTO VIEW(S).....

    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @description = 'View of V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null

-- 2) UPSERT PROTO CONFIGURATION(S).....

EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'View configuration for VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @view_name = 'VWSDEST.V_MANUAL_THIRD_SHOT_ADMINISTERED',
    @item_name = 'third_shot_administered',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1'

--VIEW BOOSTER_AND_THIRD_SHOT_ADMINISTERED
-- 1) UPSERT PROTO VIEW(S).....

    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED',
    @description = 'View of V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'View configuration for VWSDEST.V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED',
    @view_name = 'VWSDEST.V_MANUAL_BOOSTER_AND_THIRD_SHOT_ADMINISTERED',
    @item_name = 'booster_and_third_shot_administered',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1'

--VIEW V_Handmatig_BOOSTER_SHOT_PLANNED
-- 1) UPSERT PROTO VIEW(S).....

    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED',
    @description = 'View of V_MANUAL_BOOSTER_SHOT_PLANNED',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'View configuration for VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED',
    @view_name = 'VWSDEST.V_MANUAL_BOOSTER_SHOT_PLANNED',
    @item_name = 'booster_shot_planned',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1'

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS] 
    @id = null, 
    @process_name = 'PROCESS_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    @description = 'Loads the source for manual insert booster data',
    @source_name = 'SOURCE_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    @schedule = '* * * * *',
    @workflow_name = 'MANUAL INSERT BOOSTER DATA',
    @active = 1
GO

--VIEW V_MANUAL_BOOSTER_COVERAGE
-- 1) UPSERT PROTO VIEW(S).....

    EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW] 
    @id = null,
    @view_name ='VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @description = 'View of V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @last_update_name = 'DATE_OF_INSERTION_UNIX',
    @constraint_key_name = null,
    @constraint_value = null,
    @grouped_key_name = null,
    @grouped_last_update_name = null

-- 2) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = null,
    @proto_name = 'NL',
    @description = 'View configuration for VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @view_name = 'VWSDEST.V_RIVM_VACCINATIONS_BOOSTER_SHOTS_COVERAGE_NL',
    @item_name = 'booster_coverage',
    @constrained = '0',
    @grouped = '0',
    @columns = '*',
    @mapping = '=LOWER()',
    @layout_type_id = '1',
    @active = '1'

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS] 
    @id = null, 
    @process_name = 'PROCESS_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    @description = 'Loads the source for manual insert booster data',
    @source_name = 'SOURCE_LOAD_MANUAL_INSERT_BOOSTER_DATA',
    @schedule = '* * * * *',
    @workflow_name = 'MANUAL INSERT BOOSTER DATA',
    @active = 1
GO